# ΠΤΥΧΙΑΚΗ ΕΡΓΑΣΙΑ

### Μέρος Ι (EDA + MLP + CNN)

## Τσάνος Ευάγγελος

# Θέμα : "Ανίχνευση διαδικτυακών Επιθέσεων με χρήση νευρωνικών δικτύων"



---


Περιεχόμενα
--
1.Functions imports and parameters

2.EDA

- 2.1 Ελεγχος κατανομής κλάσεων και οπτικοποίηση

- 2.2 Επιπλέον έλεγχοι για null τιμές / μοναδικές τιμές επιθέσεων / ισορροπίας μεταξύ normal και επιθέσεων / Αριθμιτικά στατιστικά χαρακτηριστικά

- 2.3 Correlation matrix με hover functionality

- 2.4 Εντοπισμός outliers

- 2.5 Συσχέτιση features με target value (attack_cat)

3.Φόρτωση Δεδομένων για τους αλγόριθμους

4.Preprocessing (κοινό και για τα 2 μοντέλα)

5.MLP

6.Αξιολόγηση MLP

7.CNN

8.Αξιολόγηση CNN



---



# 1.Functions imports and parameters

In [ ]:
!pip install keras-tuner

In [ ]:
# ----------------------------------------------------------
# Basic
# ----------------------------------------------------------
import os
import pickle
import numpy as np
import pandas as pd
from scipy.stats import zscore

# ----------------------------------------------------------
# Οπτικοποίηση
# ----------------------------------------------------------
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

# ----------------------------------------------------------
# Google Drive
# ----------------------------------------------------------
from google.colab import drive

# ----------------------------------------------------------
# Προεπεξεργασία
# ----------------------------------------------------------
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import mutual_info_classif
from imblearn.over_sampling import SMOTE

# ----------------------------------------------------------
# Αξιολόγηση
# ----------------------------------------------------------
from sklearn.metrics import (
    classification_report,
    confusion_matrix,
    ConfusionMatrixDisplay,
    f1_score,
    precision_score,
    recall_score
)

# ----------------------------------------------------------
# TensorFlow / Keras
# ----------------------------------------------------------
import tensorflow as tf
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import (
    Conv1D,
    BatchNormalization,
    GlobalMaxPooling1D,
    Input,
    Dense,
    Dropout
)
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam

# ----------------------------------------------------------
# Keras Tuner
# ----------------------------------------------------------
import keras_tuner as kt

In [ ]:
# Mount Google Drive
drive.mount('/content/drive',force_remount=True)

In [ ]:
# To path που έχουμε κατεβάσει τα αρχεία στο GoogleDrive μας
base_path = '/content/drive/My Drive/UNSW-NB15/'

# Επιλογή του μεγέθους του dataset που θα δουλέψουμε για όλους τους αλγόριθμους
# Σε περίπτωση που θέλουμε να τρέξουμε tuning καλό είναι να μην πάμε με
# ολόκληρο το dataset. Ένα 30% θα εκτελεστεί σε ικανοποιητικούς χρόνους.
# Για CNN θα χρειαστεί σίγουρα GPU στο google colab
USE_TUNER = False
FRACTION = 1.0 #100%

In [ ]:
# Function που χρησιμοποιείται για την φόρτωση των αρχείων
def load_unsw_data_from_google_drive(data_path: str, fraction: float = 1.0, encoding: str = 'ISO-8859-1', random_state: int = 42):
    train_file = os.path.join(data_path, 'UNSW_NB15_training-set.csv')
    test_file = os.path.join(data_path, 'UNSW_NB15_testing-set.csv')
    try:
        df_train = pd.read_csv(train_file, encoding=encoding)
        df_test = pd.read_csv(test_file, encoding=encoding)

        if 0 < fraction < 1.0:
            df_train = df_train.sample(frac=fraction, random_state=random_state).reset_index(drop=True)
            df_test = df_test.sample(frac=fraction, random_state=random_state).reset_index(drop=True)

        print("Training set loaded. Shape:", df_train.shape)
        print("Testing set loaded. Shape:", df_test.shape)
        return df_train, df_test
    except FileNotFoundError:
        print("Τα αρχεία training ή testing δεν βρέθηκαν. Έλεγξε τη διαδρομή:", data_path)
    except Exception as e:
        print(f"Σφάλμα κατά τη φόρτωση: {e}")
    return None, None

# 2. EDA


In [ ]:
# ----------------------------------------------------------
# Φόρτωση αρχείων για EDA
# ----------------------------------------------------------
try:
    df_train, df_test = load_unsw_data_from_google_drive(base_path, fraction=1.0) #στο EDA παίρνουμε πάντα όλο το dataset
except FileNotFoundError:
    print("Τα αρχεία training ή testing δεν βρέθηκαν δεν βρέθηκαν. Παρακαλώ ελέγξτε τη διαδρομή.")
except Exception as e:
    print(f"Σφάλμα κατά τη φόρτωση των αρχείων: {e}")


if df_train is not None and df_test is not None:
    print(df_train.head())
    df_train.info()
else:
    print("Δεν φορτώθηκαν δεδομένα.")


# 2.1 Ελεγχος κατανομής κλάσεων και οπτικοποίηση

In [ ]:
# ----------------------------------------------------------
# Ανάλυση της κατανομής των κλάσεων στο Training Set (attack_cat)
# ----------------------------------------------------------
print("\nΚατανομή κλάσεων (attack_cat) στο Training Set:")
print(df_train['attack_cat'].value_counts())

# ----------------------------------------------------------
# Οπτικοποίηση της κατανομής των κλάσεων
# ----------------------------------------------------------
plt.figure(figsize=(12, 7))
sns.countplot(data=df_train, y='attack_cat', order=df_train['attack_cat'].value_counts().index, palette='viridis')
plt.title('Κατανομή Κλάσεων Επίθεσης στο Training Set')
plt.xlabel('Αριθμός Εγγραφών')
plt.ylabel('Κατηγορία Επίθεσης')
plt.grid(axis='x', linestyle='--', alpha=0.7)
plt.show()

# ----------------------------------------------------------
# Υπολογισμός ποσοστών για καλύτερη κατανόηση της ανισορροπίας
# ----------------------------------------------------------
print("\nΠοσοστά Κατανομής Κλάσεων (attack_cat) στο Training Set:")
print(df_train['attack_cat'].value_counts(normalize=True) * 100)

# ----------------------------------------------------------
# Επανάληψη για το Testing Set - για να δούμε την
# ανισοκατανομή και στο τεστ σετ
# ----------------------------------------------------------
print("\nΚατανομή κλάσεων (attack_cat) στο Testing Set:")
print(df_test['attack_cat'].value_counts())

plt.figure(figsize=(12, 7))
sns.countplot(data=df_test, y='attack_cat', order=df_test['attack_cat'].value_counts().index, palette='magma')
plt.title('Κατανομή Κλάσεων Επίθεσης στο Testing Set')
plt.xlabel('Αριθμός Εγγραφών')
plt.ylabel('Κατηγορία Επίθεσης')
plt.grid(axis='x', linestyle='--', alpha=0.7)
plt.show()

# 2.2 Επιπλέον έλεγχοι για null τιμές / μοναδικές τιμές επιθέσεων / ισορροπίας μεταξύ normal και επιθέσεων / Αριθμιτικά στατιστικά χαρακτηριστικά

In [ ]:
#Έλεγχος για null τιμές:
print("\nΑριθμός μηδενικών τιμών ανά στήλη:")
print(df_train.isnull().sum())

#Έλεγχος μοναδικών τιμών στην ετικέτα:
print("Μοναδικές κατηγορίες επιθέσεων:")
print(df_train['attack_cat'].unique())

#Έλεγχος για ισορροπία μεταξύ "normal" και επιθέσεων:
print(df_train['label'].value_counts())  # 1=attack 0=normal

#Έλεγχος για ισορροπία μεταξύ τύπων επιθέσεων
print(df_train['attack_cat'].value_counts())  # 1=attack 0=normal

#Αριθμητικά στατιστικά χαρακτηριστικά
print(df_train.describe())

 # 2.3 Correlation matrix με hover functionality

In [ ]:
# Αντιγραφή και Label Encoding
df_encoded = df_train.copy()
le = LabelEncoder()
df_encoded['attack_cat'] = le.fit_transform(df_encoded['attack_cat'])

# Αφαίρεση του id
if 'ï»¿id' in df_encoded.columns:
    df_encoded.drop(columns=['ï»¿id'], inplace=True)

# Επιλογή μόνο αριθμητικών
numeric_df = df_encoded.select_dtypes(include=['int64', 'float64'])

# Υπολογισμός του correlation matrix
correlation_matrix = numeric_df.corr()

# Δημιουργία interactive heatmap με Plotly
fig = px.imshow(
    correlation_matrix,
    text_auto=".2f",
    color_continuous_scale='RdBu_r',
    title='Interactive Correlation Matrix'
)

fig.update_layout(
    width=1000,
    height=1000
)

fig.show()

# 2.4 Εντοπισμός outliers


In [ ]:
def detect_outliers(df, z_thresh=3.0, iqr_multiplier=1.5):
    # Εντοπισμός αριθμητικών χαρακτηριστικών
    numeric_cols = df.select_dtypes(include=['float64', 'int64']).columns
    df_numeric = df[numeric_cols]

    # Z-score
    z_scores = np.abs(zscore(df_numeric))
    z_outliers = (z_scores > z_thresh)
    z_counts = z_outliers.sum(axis=0)

    # IQR
    Q1 = df_numeric.quantile(0.25)
    Q3 = df_numeric.quantile(0.75)
    IQR = Q3 - Q1
    iqr_outliers = ((df_numeric < (Q1 - iqr_multiplier * IQR)) |
                    (df_numeric > (Q3 + iqr_multiplier * IQR)))
    iqr_counts = iqr_outliers.sum()

    # Συγκεντρωτικός πίνακας
    result = pd.DataFrame({
        'Z_Outliers': z_counts,
        'IQR_Outliers': iqr_counts,
        'Total_Rows': len(df),
        'Z_%': (z_counts / len(df)) * 100,
        'IQR_%': (iqr_counts / len(df)) * 100
    })

    # κατηγοριοποίηση
    def suggest_action(pct):
        if pct > 10:
            return 'Πιθανός log-transform ή clipping'
        elif pct > 1:
            return 'Μέτριοι outliers - επανεξέταση'
        else:
            return 'Δεν χρειάζεται ενέργεια'

    result['Πρόταση'] = result['Z_%'].apply(suggest_action)

    return result.sort_values('Z_%', ascending=False)


outlier_report = detect_outliers(df_train)
display(outlier_report)

# 2.5 Συσχέτιση features με target value (attack_cat)

In [ ]:
# Αν υπάρχουν χαρακτηριστικά που επηρρεάζουν πολύ λίγο θα μπορούσαμε να μην τα
# χρησιμοποιήσουμε στα μοντέλα μας. Έτσι θα μειώναμε και το dataset. Παρόλα
# αυτά για το δικό μας παράδειγμα δεν θα κάνουμε κάποια στήλη exclude.
X = numeric_df.drop(columns=['label'])
y = df_encoded['attack_cat']

mi = mutual_info_classif(X, y, discrete_features='auto')
mi_scores = pd.Series(mi, index=X.columns).sort_values(ascending=False)

plt.figure(figsize=(10, 6))
mi_scores.plot(kind='bar')
plt.title('Mutual Information των Χαρακτηριστικών με το Target')
plt.show()

# 3. Φόρτωση Δεδομένων για τους αλγόριθμους

In [ ]:
# ----------------------------------------------------------
# Φόρτωση αρχείων για τα μοντέλα μας
# εδώ παίζει ρόλο η παράμετρος FRACTION !!!
# ----------------------------------------------------------
try:
    df_train, df_test = load_unsw_data_from_google_drive(base_path, fraction=FRACTION)
except FileNotFoundError:
    print("Τα αρχεία training-set.csv ή testing-set.csv δεν βρέθηκαν. Παρακαλώ ελέγξτε τη διαδρομή.")
except Exception as e:
    print(f"Σφάλμα κατά τη φόρτωση των αρχείων: {e}")


if df_train is not None and df_test is not None:
    print(df_train.head())
    df_train.info()
else:
    print("Δεν φορτώθηκαν δεδομένα.")

# 4. Preprocessing (κοινό και για τα 2 μοντέλα)

In [ ]:
# ----------------------------------------------------------
# Καθαρισμός δεδομένων & Επιλογή χαρακτηριστικών
# ----------------------------------------------------------
features_to_exclude = ['ï»¿id', 'label', 'attack_cat']
selected_features = [col for col in df_train.columns if col not in features_to_exclude]

X_train_raw = df_train[selected_features].copy()
X_test_raw  = df_test[selected_features].copy()

# ----------------------------------------------------------
# Εντοπισμός & Αντικατάσταση σπάνιων κατηγοριών
# ----------------------------------------------------------
categorical_columns = X_train_raw.select_dtypes(include='object').columns
rare_threshold = 100  # Ελάχιστες εμφανίσεις για να κρατήσουμε την τιμή ως "μη σπάνια"

for col in categorical_columns:
    value_counts = X_train_raw[col].value_counts()
    rare_values = value_counts[value_counts < rare_threshold].index

    X_train_raw[col] = X_train_raw[col].replace(rare_values, 'rare')
    X_test_raw[col]  = X_test_raw[col].replace(rare_values, 'rare')

# ----------------------------------------------------------
# Label Encoding των στόχων (attack_cat)
# ----------------------------------------------------------
label_encoder = LabelEncoder()
y_train_int = label_encoder.fit_transform(df_train['attack_cat'])
y_test_int  = label_encoder.transform(df_test['attack_cat'])

num_classes = len(label_encoder.classes_)
print(f"Κλάσεις ταξινόμησης: {label_encoder.classes_}")

# Αποθήκευση encoder για μελλοντική χρήση (π.χ. API)
with open("attack_cat_encoder.pkl", "wb") as f:
    pickle.dump(label_encoder, f)

# ----------------------------------------------------------
# One-Hot Encoding των κατηγορικών γνωρισμάτων
# ----------------------------------------------------------
X_train_ohe = pd.get_dummies(X_train_raw, columns=categorical_columns)
X_test_ohe  = pd.get_dummies(X_test_raw,  columns=categorical_columns)

# Ευθυγράμμιση κοινών χαρακτηριστικών μεταξύ train και test
common_features = X_train_ohe.columns.intersection(X_test_ohe.columns)
X_train_aligned = X_train_ohe[common_features]
X_test_aligned  = X_test_ohe[common_features]

# ----------------------------------------------------------
# Κανονικοποίηση γνωρισμάτων (MinMax Scaling)
# ----------------------------------------------------------
scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train_aligned)
X_test_scaled  = scaler.transform(X_test_aligned)

# ----------------------------------------------------------
# Train/Validation Split με stratify
# ----------------------------------------------------------
X_train_final, X_val_final, y_train_final, y_val_final = train_test_split(
    X_train_scaled,
    y_train_int,
    test_size=0.2,
    stratify=y_train_int,
    random_state=42
)

# ----------------------------------------------------------
# Αντιμετώπιση Imbalance με SMOTE στο training set
# ----------------------------------------------------------
smote = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train_final, y_train_final)

# ----------------------------------------------------------
# One-Hot Encoding στα labels για MLP & CNN
# ----------------------------------------------------------
y_train_ohe = to_categorical(y_train_resampled, num_classes=num_classes)
y_val_ohe   = to_categorical(y_val_final,     num_classes=num_classes)
y_test_ohe  = to_categorical(y_test_int,      num_classes=num_classes)

# ----------------------------------------------------------
# Έλεγχος shapes
# ----------------------------------------------------------
print(f"X_train shape: {X_train_resampled.shape}")
print(f"X_val shape:   {X_val_final.shape}")
print(f"X_test shape:  {X_test_scaled.shape}")
print(f"y_train shape: {y_train_ohe.shape}")
print(f"y_val shape:   {y_val_ohe.shape}")
print(f"y_test shape:  {y_test_ohe.shape}")


# 5. MLP

In [ ]:
# ----------------------------------------------------------
# MLP με 3 hidden layers + Keras Tuner
# ----------------------------------------------------------

def build_model(hp):
    model = Sequential()

    model.add(Dense(
        units=hp.Int('dense_units_1', 128, 512, step=64),
        activation='relu',
        input_shape=(X_train_resampled.shape[1],)
    ))
    model.add(Dropout(hp.Float('dropout_1', 0.2, 0.5, step=0.1)))

    model.add(Dense(
        units=hp.Int('dense_units_2', 64, 256, step=32),
        activation='relu'
    ))
    model.add(Dropout(hp.Float('dropout_2', 0.2, 0.5, step=0.1)))

    model.add(Dense(
        units=hp.Int('dense_units_3', 32, 128, step=32),
        activation='relu'
    ))
    model.add(Dropout(hp.Float('dropout_3', 0.2, 0.5, step=0.1)))

    model.add(Dense(num_classes, activation='softmax'))

    learning_rate = hp.Choice('learning_rate', [0.01, 0.001, 0.0001])
    model.compile(
        optimizer=Adam(learning_rate=learning_rate),
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )
    return model


# Default parameters από το tuner
default_params = {
    'dense_units_1': 256,
    'dropout_1': 0.3,
    'dense_units_2': 128,
    'dropout_2': 0.3,
    'dense_units_3': 64,
    'dropout_3': 0.3,
    'learning_rate': 0.001
}


def build_default_model():
    model = Sequential()

    model.add(Dense(default_params['dense_units_1'], activation='relu',
                    input_shape=(X_train_resampled.shape[1],)))
    model.add(Dropout(default_params['dropout_1']))

    model.add(Dense(default_params['dense_units_2'], activation='relu'))
    model.add(Dropout(default_params['dropout_2']))

    model.add(Dense(default_params['dense_units_3'], activation='relu'))
    model.add(Dropout(default_params['dropout_3']))

    model.add(Dense(num_classes, activation='softmax'))

    model.compile(optimizer=Adam(learning_rate=default_params['learning_rate']),
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    return model


# Early stopping
early_stop = EarlyStopping(
    monitor='val_loss',
    patience=10,
    restore_best_weights=True
)


if USE_TUNER:
    tuner = kt.Hyperband(
        build_model,
        objective='val_accuracy',
        max_epochs=50,
        factor=3,
        directory='kt_dir',
        project_name='mlp_unsw_3hidden'
    )

    tuner.search(
        X_train_resampled,
        y_train_ohe,
        epochs=50,
        batch_size=128,
        validation_data=(X_val_final, y_val_ohe),
        callbacks=[early_stop]
    )

    best_hps = tuner.get_best_hyperparameters(1)[0]
    print(f"""
    ΒΕΛΤΙΣΤΕΣ ΥΠΕΡΠΑΡΑΜΕΤΡΟΙ:
    - dense_units_1: {best_hps.get('dense_units_1')}
    - dense_units_2: {best_hps.get('dense_units_2')}
    - dense_units_3: {best_hps.get('dense_units_3')}
    - dropout_1: {best_hps.get('dropout_1')}
    - dropout_2: {best_hps.get('dropout_2')}
    - dropout_3: {best_hps.get('dropout_3')}
    - learning_rate: {best_hps.get('learning_rate')}
    """)

    tuner.results_summary()
    best_model = tuner.hypermodel.build(best_hps)
else:
    print("Χρήση DEFAULT τιμών χωρίς Keras Tuner")
    best_model = build_default_model()


# Εκπαίδευση του τελικού μοντέλου
history = best_model.fit(
    X_train_resampled,
    y_train_ohe,
    epochs=50,
    batch_size=128,
    validation_data=(X_val_final, y_val_ohe),
    callbacks=[early_stop]
)

# Αξιολόγηση στο test set
test_loss, test_accuracy = best_model.evaluate(X_test_scaled, y_test_ohe)
print(f"Test accuracy: {test_accuracy:.4f}")

# Αποθήκευση τελικού μοντέλου
best_model.save("best_mlp_unsw_3hidden.h5")


## 6. Αξιολόγηση MLP

In [ ]:
# ---------------------------
# Διαγράμματα εκπαίδευσης
# ---------------------------
plt.figure()
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title("Loss ανά Εποχή")
plt.xlabel("Εποχές")
plt.ylabel("Loss")
plt.legend()
plt.grid(True)
plt.show()

plt.figure()
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title("Ακρίβεια ανά Εποχή")
plt.xlabel("Εποχές")
plt.ylabel("Accuracy")
plt.legend()
plt.grid(True)
plt.show()

# ---------------------------
# Προβλέψεις στο test set
# ---------------------------
y_test_predictions_proba = best_model.predict(X_test_scaled)
y_test_predictions = np.argmax(y_test_predictions_proba, axis=1)
y_test_true = np.argmax(y_test_ohe, axis=1)

# ---------------------------
# Classification Report
# ---------------------------
print("\nClassification Report:")
print(classification_report(
    y_test_true,
    y_test_predictions,
    target_names=label_encoder.classes_,
    digits=4
))

# ---------------------------
# Confusion Matrix
# ---------------------------
conf_matrix = confusion_matrix(y_test_true, y_test_predictions)
disp = ConfusionMatrixDisplay(confusion_matrix=conf_matrix, display_labels=label_encoder.classes_)
disp.plot(xticks_rotation=90, cmap='Blues')
plt.title("Confusion Matrix")
plt.show()

# ---------------------------
# Επιπλέον Μετρικά
# ---------------------------
print(f"\nMacro F1 Score:      {f1_score(y_test_true, y_test_predictions, average='macro'):.4f}")
print(f"Weighted F1 Score:   {f1_score(y_test_true, y_test_predictions, average='weighted'):.4f}")
print(f"Micro F1 Score:      {f1_score(y_test_true, y_test_predictions, average='micro'):.4f}")
print(f"Macro Precision:     {precision_score(y_test_true, y_test_predictions, average='macro'):.4f}")
print(f"Macro Recall:        {recall_score(y_test_true, y_test_predictions, average='macro'):.4f}")


#7. CNN



In [ ]:
# ----------------------------------------------------------
# Extra Preprocessing Βήμα για CNN
# ----------------------------------------------------------

# Reshape 3D: (samples, features, 1) για χρήση σε Conv1D
X_train_cnn = X_train_resampled.reshape(-1, X_train_resampled.shape[1], 1)
X_val_cnn   = X_val_final.reshape(-1, X_val_final.shape[1], 1)
X_test_cnn  = X_test_scaled.reshape(-1, X_test_scaled.shape[1], 1)



In [ ]:
# ----------------------------------------------------------
# Custom Model με τις Καλύτερες Τιμές (από tuner)
# ----------------------------------------------------------
def build_custom_model():
    model = Sequential()
    model.add(Input(shape=(X_train_cnn.shape[1], 1)))

    # Καλύτερες τιμές που έβγαλε ο tuner
    filters = 256
    kernel_size = 7
    dropout_rate = 0.3
    dense_units = 192
    learning_rate = 0.001

    # CNN Αρχιτεκτονική
    model.add(Conv1D(filters=filters, kernel_size=kernel_size, activation='relu', padding='same'))
    model.add(BatchNormalization())
    model.add(Conv1D(filters=filters, kernel_size=kernel_size, activation='relu', padding='same'))
    model.add(BatchNormalization())
    model.add(GlobalMaxPooling1D())
    model.add(Dense(dense_units, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(dropout_rate))
    model.add(Dense(num_classes, activation='softmax'))

    model.compile(
        optimizer=Adam(learning_rate=learning_rate),
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )

    return model


# ----------------------------------------------------------
# CNN Model με Tuner
# ----------------------------------------------------------
def build_model(hp):
    model = Sequential()
    model.add(Input(shape=(X_train_cnn.shape[1], 1)))

    # Υπερπαράμετροι
    filters = hp.Int("filters", min_value=64, max_value=256, step=64)
    kernel_size = hp.Choice("kernel_size", values=[3, 5, 7])
    dropout_rate = hp.Float("dropout", min_value=0.1, max_value=0.4, step=0.1)
    dense_units = hp.Int("dense_units", min_value=64, max_value=256, step=64)
    learning_rate = hp.Float("lr", min_value=0.0001, max_value=0.01, sampling="log")

    model.add(Conv1D(filters=filters, kernel_size=kernel_size, activation='relu', padding='same'))
    model.add(BatchNormalization())
    model.add(Conv1D(filters=filters, kernel_size=kernel_size, activation='relu', padding='same'))
    model.add(BatchNormalization())
    model.add(GlobalMaxPooling1D())
    model.add(Dense(dense_units, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(dropout_rate))
    model.add(Dense(num_classes, activation='softmax'))

    model.compile(
        optimizer=Adam(learning_rate=learning_rate),
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )

    return model


# ----------------------------------------------------------
# Keras Tuner Setup
# ----------------------------------------------------------
tuner = kt.RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=30,
    executions_per_trial=1,
    directory='kt_dir',
    project_name='cnn_unsw'
)

# Callbacks
early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, min_lr=1e-6)

# ----------------------------------------------------------
# Εκπαίδευση
# ----------------------------------------------------------
if USE_TUNER:
    tuner.search(
        X_train_cnn, y_train_ohe,
        epochs=50,
        batch_size=128,
        validation_data=(X_val_cnn, y_val_ohe),
        callbacks=[early_stop, reduce_lr]
    )
    best_model = tuner.get_best_models(num_models=1)[0]
else:
    print("Χρήση custom CNN με default τιμές (χωρίς tuner)")
    model = build_custom_model()
    history = model.fit(
        X_train_cnn, y_train_ohe,
        validation_data=(X_val_cnn, y_val_ohe),
        epochs=50,
        batch_size=128,
        callbacks=[early_stop, reduce_lr]
    )
    best_model = model

# ----------------------------------------------------------
# Περίληψη & Αξιολόγηση
# ----------------------------------------------------------
best_model.summary()

test_loss, test_accuracy = best_model.evaluate(X_test_cnn, y_test_ohe)
print(f"Test accuracy: {test_accuracy:.4f}")

# ----------------------------------------------------------
# Αποθήκευση μοντέλου
# ----------------------------------------------------------
best_model.save("cnn_tabular_unsw_best.h5")


## 8. Αξιολόγηση CNN

In [ ]:
# ----------------------------------------------------------
# Διαγράμματα Εκπαίδευσης (Loss & Accuracy)
# ----------------------------------------------------------
plt.figure()
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title("Loss ανά Εποχή (CNN)")
plt.xlabel("Εποχές")
plt.ylabel("Loss")
plt.legend()
plt.grid(True)
plt.show()

plt.figure()
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title("Ακρίβεια ανά Εποχή (CNN)")
plt.xlabel("Εποχές")
plt.ylabel("Accuracy")
plt.legend()
plt.grid(True)
plt.show()

# ----------------------------------------------------------
# Προβλέψεις στο test set
# ----------------------------------------------------------
y_test_pred_probs = best_model.predict(X_test_cnn)
y_test_pred = np.argmax(y_test_pred_probs, axis=1)
y_test_true = np.argmax(y_test_ohe, axis=1)

# ----------------------------------------------------------
# Classification Report
# ----------------------------------------------------------
print("\nClassification Report (CNN):")
print(classification_report(
    y_test_true,
    y_test_pred,
    target_names=label_encoder.classes_,
    digits=4
))

# ----------------------------------------------------------
# Confusion Matrix
# ----------------------------------------------------------
conf_matrix = confusion_matrix(y_test_true, y_test_pred)
disp = ConfusionMatrixDisplay(confusion_matrix=conf_matrix, display_labels=label_encoder.classes_)
disp.plot(xticks_rotation=90, cmap='Blues')
plt.title("Confusion Matrix (CNN)")
plt.show()

# ----------------------------------------------------------
# Επιπλέον Μετρικά
# ----------------------------------------------------------
print(f"\nMacro F1 Score:      {f1_score(y_test_true, y_test_pred, average='macro'):.4f}")
print(f"Weighted F1 Score:   {f1_score(y_test_true, y_test_pred, average='weighted'):.4f}")
print(f"Micro F1 Score:      {f1_score(y_test_true, y_test_pred, average='micro'):.4f}")
print(f"Macro Precision:     {precision_score(y_test_true, y_test_pred, average='macro'):.4f}")
print(f"Macro Recall:        {recall_score(y_test_true, y_test_pred, average='macro'):.4f}")
